# 📚 Natural Language Processing: Spelling Correction
### BITS Pilani - NLP Applications Tutorial
This notebook provides hands-on implementation of spelling correction concepts including:
- Basic spell checking
- Edit distance calculation
- Noisy channel model
- Context-aware corrections
- Enhanced Error Types Handler
- Technical Terms Handler
- Evaluation Framework

## 1. Setup and Required Libraries

First, let's install and import the required libraries:

In [1]:
!pip install nltk

import numpy as np
import pandas as pd
from collections import Counter
import re
from typing import List, Dict, Set, Tuple
import nltk
from nltk.corpus import words
nltk.download('words')

print("Setup completed!")



Setup completed!


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


## 2. Basic Spell Checking Functions

Let's implement the core functions needed for spell checking:

In [14]:
class SpellChecker:
    def __init__(self):
        # Initialize vocabulary using NLTK words
        self.vocabulary = set(words.words())

    def is_word(self, word: str) -> bool:
        """Check if a word exists in the vocabulary."""
        return word.lower() in self.vocabulary

    def edit_distance(self, s1: str, s2: str) -> int:
        """Calculate Damerau-Levenshtein distance between two strings."""
        len_s1, len_s2 = len(s1), len(s2)
        dp = [[0] * (len_s2 + 1) for _ in range(len_s1 + 1)]

        # Initialize first row and column
        for i in range(len_s1 + 1):
            dp[i][0] = i
        for j in range(len_s2 + 1):
            dp[0][j] = j

        for i in range(1, len_s1 + 1):
            for j in range(1, len_s2 + 1):
                cost = 0 if s1[i-1] == s2[j-1] else 1
                dp[i][j] = min(
                    dp[i-1][j] + 1,      # deletion
                    dp[i][j-1] + 1,      # insertion
                    dp[i-1][j-1] + cost  # substitution
                )
                # Handle transposition
                if (i > 1 and j > 1 and s1[i-1] == s2[j-2] and s1[i-2] == s2[j-1]):
                    dp[i][j] = min(dp[i][j], dp[i-2][j-2] + 1)

        return dp[len_s1][len_s2]

    def generate_candidates(self, word: str, max_distance: int = 2) -> List[str]:
        """Generate possible corrections for a word."""
        return [w for w in self.vocabulary
                if self.edit_distance(word.lower(), w.lower()) <= max_distance]

# Create instance
spell_checker = SpellChecker()
print("Spell checker initialized!")

Spell checker initialized!


## 3. Testing Basic Spell Checking

Let's test our spell checker with some examples:

In [15]:
test_words = ["happyness", "langauge", "speling", "computr"]

for word in test_words:
    candidates = spell_checker.generate_candidates(word)
    print(f"\nMisspelled word: {word}")
    print(f"Suggested corrections: {', '.join(candidates[:5])}")


Misspelled word: happyness
Suggested corrections: happiless, yappiness, happiness, nappiness, sappiness

Misspelled word: langauge
Suggested corrections: language, languaged, langrage, slanguage

Misspelled word: speling
Suggested corrections: seeking, shelling, keeling, styling, smiling

Misspelled word: computr
Suggested corrections: comport, compete, commuter, compeer, compote


## 4. Implementing Noisy Channel Model

Now let's implement the noisy channel model for better correction suggestions:

In [16]:
class NoiseChannelSpellChecker(SpellChecker):
    def __init__(self):
        super().__init__()
        # Add word frequency information
        self.word_freq = Counter()

    def train(self, text: str):
        """Train the model on a text corpus."""
        words = re.findall(r'\w+', text.lower())
        self.word_freq.update(words)
        self.total_words = sum(self.word_freq.values())

    def P_word(self, word: str) -> float:
        """Calculate prior probability of word."""
        return (self.word_freq[word.lower()] + 1) / (self.total_words + len(self.vocabulary))

    def P_error(self, error: str, word: str) -> float:
        """Calculate error probability P(error|word)."""
        distance = self.edit_distance(error.lower(), word.lower())
        return np.exp(-distance)

    def correct_word(self, error: str) -> str:
        """Find most likely correction."""
        if self.is_word(error):
            return error

        candidates = self.generate_candidates(error)
        if not candidates:
            return error

        # Find correction with highest probability
        return max(candidates,
                  key=lambda w: self.P_error(error, w) * self.P_word(w))

# Create and train noisy channel model
noise_checker = NoiseChannelSpellChecker()

# Sample training text
training_text = """
Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence
concerned with the interactions between computers and human language.
"""

noise_checker.train(training_text)
print("Noisy channel model trained!")

Noisy channel model trained!


## 5. Evaluation and Testing

Let's test our noisy channel spell checker:

In [5]:
test_cases = [
    "naturel",
    "languge",
    "computr",
    "artifical"
]

print("Spell Correction Results:")
print("-" * 40)
for word in test_cases:
    correction = noise_checker.correct_word(word)
    print(f"Original: {word:15} → Correction: {correction}")

Spell Correction Results:
----------------------------------------
Original: naturel         → Correction: natural
Original: languge         → Correction: language
Original: computr         → Correction: computer
Original: artifical       → Correction: artificial


# 6. Context-Aware Spell Checker

In [17]:
class ContextAwareSpellChecker(NoiseChannelSpellChecker):
    def __init__(self):
        super().__init__()
        self.bigram_counts = Counter()

    def train(self, text: str):
        """Train both word frequencies and bigram counts."""
        super().train(text)
        # Create bigrams from text
        words = re.findall(r'\w+', text.lower())
        self.bigrams = list(zip(words[:-1], words[1:]))
        self.bigram_counts.update(self.bigrams)

    def P_context(self, prev_word: str, word: str, next_word: str) -> float:
        """Calculate probability based on surrounding context."""
        prev_bigram = (prev_word.lower(), word.lower())
        next_bigram = (word.lower(), next_word.lower())

        # Combine previous and next bigram probabilities
        prev_count = self.bigram_counts[prev_bigram] + 1
        next_count = self.bigram_counts[next_bigram] + 1
        total_bigrams = sum(self.bigram_counts.values()) + len(self.vocabulary)**2

        return (prev_count * next_count) / (total_bigrams * total_bigrams)

    def correct_text(self, text: str) -> str:
        """Correct entire text considering context."""
        words = re.findall(r'\w+', text)
        corrected_words = []

        for i in range(len(words)):
            prev_word = words[i-1] if i > 0 else "<START>"
            next_word = words[i+1] if i < len(words)-1 else "<END>"

            if not self.is_word(words[i]):
                candidates = self.generate_candidates(words[i])
                if candidates:
                    # Consider both error probability and context
                    best_word = max(candidates,
                                  key=lambda w: (self.P_error(words[i], w) *
                                               self.P_word(w) *
                                               self.P_context(prev_word, w, next_word)))
                    corrected_words.append(best_word)
                else:
                    corrected_words.append(words[i])
            else:
                corrected_words.append(words[i])

        return ' '.join(corrected_words)

# 7. Test context-aware spell checker

In [7]:
context_checker = ContextAwareSpellChecker()
context_checker.train("""
    The quick brown fox jumps over the lazy dog.
    Natural language processing helps computers understand human language.
    Machine learning models can process large amounts of text data.
""")

test_text = "The quik brown fox jumps ovr the lasy dog"
corrected_text = context_checker.correct_text(test_text)
print(f"Original: {test_text}")
print(f"Corrected: {corrected_text}")

Original: The quik brown fox jumps ovr the lasy dog
Corrected: The quick brown fox mumps over the lazy dog


# 8. Enhanced Error Types Handler

In [8]:
class EnhancedSpellChecker(ContextAwareSpellChecker):
    def generate_candidates_enhanced(self, word: str) -> Set[str]:
        """Generate candidates with various error types."""
        candidates = set()
        letters = 'abcdefghijklmnopqrstuvwxyz'
        word = word.lower()

        # Original edit distance candidates
        candidates.update(self.generate_candidates(word))

        # 1. Transpositions
        for i in range(len(word)-1):
            transposed = word[:i] + word[i+1] + word[i] + word[i+2:]
            if self.is_word(transposed):
                candidates.add(transposed)

        # 2. Phonetic similarities
        phonetic_rules = {
            'ph': 'f', 'ght': 't', 'kn': 'n', 'wh': 'w',
            'ing': 'in', 'ed': 't', 'tion': 'shun'
        }

        for pattern, replacement in phonetic_rules.items():
            if pattern in word:
                phonetic = word.replace(pattern, replacement)
                if self.is_word(phonetic):
                    candidates.add(phonetic)

        # 3. Common typos
        keyboard_neighbors = {
            'a': 'qwsz', 'b': 'vghn', 'c': 'xdfv', 'd': 'sfxc',
            'e': 'wrsdf', 'f': 'dcvgr', 'g': 'fvbht', 'h': 'gbynj',
            'i': 'ujko', 'j': 'hunkm', 'k': 'jmlo', 'l': 'kop',
            'm': 'njk', 'n': 'bhjm', 'o': 'iklp', 'p': 'ol',
            'q': 'wa', 'r': 'edft', 's': 'awdxz', 't': 'rfgy',
            'u': 'yhji', 'v': 'cfgb', 'w': 'qase', 'x': 'zsdc',
            'y': 'tghu', 'z': 'asx'
        }

        for i, char in enumerate(word):
            if char in keyboard_neighbors:
                for neighbor in keyboard_neighbors[char]:
                    typo = word[:i] + neighbor + word[i+1:]
                    if self.is_word(typo):
                        candidates.add(typo)

        return candidates

    def correct_word(self, error: str) -> str:
        """Find most likely correction using enhanced candidate generation."""
        if self.is_word(error):
            return error

        candidates = self.generate_candidates_enhanced(error)
        if not candidates:
            return error

        return max(candidates,
                  key=lambda w: self.P_error(error, w) * self.P_word(w))

# 9. Test enhanced spell checker

In [9]:
enhanced_checker = EnhancedSpellChecker()
enhanced_checker.train("""
    The quick brown fox jumps over the lazy dog.
    Physics and chemistry are important sciences.
    Knowledge is power in the modern world.
""")

test_words = ["phisics", "kemistry", "nowledge", "recieved"]
for word in test_words:
    correction = enhanced_checker.correct_word(word)
    print(f"Original: {word:10} → Correction: {correction}")

Original: phisics    → Correction: physics
Original: kemistry   → Correction: chemistry
Original: nowledge   → Correction: knowledge
Original: recieved   → Correction: received


 # 10. Technical Terms Handler

In [10]:
class TechnicalSpellChecker(EnhancedSpellChecker):
    def __init__(self):
        super().__init__()
        self.technical_terms = set()
        self.proper_nouns = set()

    def add_technical_vocabulary(self, domain: str, terms: List[str]):
        """Add domain-specific technical terms."""
        self.technical_terms.update(term.lower() for term in terms)

    def add_proper_nouns(self, nouns: List[str]):
        """Add proper nouns to vocabulary."""
        self.proper_nouns.update(noun.lower() for noun in nouns)

    def is_word(self, word: str) -> bool:
        """Check if word exists in any vocabulary."""
        word = word.lower()
        return (super().is_word(word) or
                word in self.technical_terms or
                word in self.proper_nouns)

    def suggest_technical_terms(self, word: str) -> List[str]:
        """Suggest corrections from technical vocabulary."""
        return [term for term in self.technical_terms
                if self.edit_distance(word.lower(), term) <= 2]

# 11. Test technical spell checker

In [11]:
tech_checker = TechnicalSpellChecker()

# First train with some base text
tech_checker.train("""
    Programming languages are essential tools for software development.
    Machine learning models help in processing data efficiently.
    Cloud computing enables scalable solutions for businesses.
    Google Cloud is a popular cloud service provider.
    sklearn and tensorflow are popular machine learning library.
""")

# Add technical terms
tech_terms = [
    "Python", "JavaScript", "React", "TensorFlow",
    "NumPy", "pandas", "sklearn", "PyTorch"
]
tech_checker.add_technical_vocabulary("programming", tech_terms)

# Add proper nouns
proper_nouns = ["Google", "Microsoft", "Amazon", "Facebook"]
tech_checker.add_proper_nouns(proper_nouns)

# Test the spell checker
test_technical = ["pythn", "tensorflow", "googel", "sklaern"]
for word in test_technical:
    correction = tech_checker.correct_word(word)
    print(f"Technical term: {word:10} → Correction: {correction}")

Technical term: pythn      → Correction: python
Technical term: tensorflow → Correction: tensorflow
Technical term: googel     → Correction: google
Technical term: sklaern    → Correction: sklearn


# 12. Evaluation Framework

In [12]:
class SpellCheckerEvaluator:
    def __init__(self, spell_checker):
        self.spell_checker = spell_checker

    def evaluate(self, test_cases: List[Tuple[str, str]]) -> Dict[str, float]:
        """
        Evaluate spell checker performance.
        test_cases: List of (error, correct) word pairs
        """
        total = len(test_cases)
        correct = 0
        false_positives = 0
        false_negatives = 0

        for error, truth in test_cases:
            prediction = self.spell_checker.correct_word(error)

            if prediction == truth:
                correct += 1
            else:
                if self.spell_checker.is_word(error) and error != truth:
                    false_negatives += 1
                if prediction != truth:
                    false_positives += 1

        precision = correct / (correct + false_positives) if (correct + false_positives) > 0 else 0
        recall = correct / (correct + false_negatives) if (correct + false_negatives) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return {
            "accuracy": correct / total,
            "precision": precision,
            "recall": recall,
            "f1_score": f1
        }


# 13. Test evaluation framework

In [13]:
test_cases = [
    ("happyness", "happiness"),
    ("recieve", "receive"),
    ("independant", "independent"),
    ("occured", "occurred"),
    ("definately", "definitely")
]

evaluator = SpellCheckerEvaluator(enhanced_checker)
metrics = evaluator.evaluate(test_cases)

print("\nEvaluation Results:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.3f}")


Evaluation Results:
accuracy: 0.600
precision: 0.600
recall: 1.000
f1_score: 0.750
